# Título   Crear clase ternaria con duckdb
## Autor: Jose Chelquer
## Fecha de última modificación: 23/10/2025
## Descripción:

Crear clase ternaria usando duckdb

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [7]:
#usar_gdrive=True    # Poner en true si se va a ejecutar con archivos de google drive en máquinas locales
usar_gdrive=False

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [35]:
#dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
dataset_path = '~/buckets/b1/datasets'
#dataset_file='competencia_01.csv'
dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [3]:
output_file='competencia_02_duck.csv.gz'
duckdb_file='competencia_02.db'

## Procesos

### Paquetes necesarios

In [4]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [5]:
import os
import pandas as pd
import duckdb

In [8]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [29]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
duckdb_path=os.path.join(dataset_path, duckdb_file)

conn = duckdb.connect(duckdb_path)


In [30]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes: int, meses: int) -> int:
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm



In [13]:

# Registrar la función UDF en DuckDB
conn.create_function('sig_mes', sig_mes)

## Leer datos en tabla

In [14]:
def do_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  print (resultado)

In [16]:
# Leer el CSV y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
sql = f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}', COMPRESSION='GZIP')"
print (sql)
conn.execute(sql)



CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('~/buckets/b1/datasets/competencia_02_crudo.csv.gz', COMPRESSION='GZIP')


In [17]:
# Verificar si la tabla fue creada correctamente
result = conn.execute("SELECT * FROM competencia_02 LIMIT 1").fetchall()
print(result)
# Contar los registros en la tabla
count = conn.execute("SELECT COUNT(*) FROM competencia_02").fetchone()[0]
print (f'Count:{count}')



[(249221109, 201901, 1, 0, 1, 59, 276, 7597.55, 47433.58, 5654.59, -915.57, 2571.97, 9, 1, 1, 0.0, -317.98, 2, 24068.63, 0.0, 268675.63, 1, 280992.59, 3, 0, 0.0, 1, 23, 31249.45, 1, 5, 9333.88, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0, 0, 1, 1, 0, 0, 0.0, 0.0, 0, 5, 15333.25, 6, 19820.48, 4, 8786.59, 0, 0.0, 6, 14612.56, 0, 0.0, 0, 0.0, 0, 0.0, 1, 1199.97, 18, 5654.59, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 1, 5161.2, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0, 1, 71, 1, 1, 4, 2, 0, 0, 0.0, 0, 0.0, 164, None, None, 0, 0, 209004.84, -1307, None, 9333.88, 9333.88, 0.0, 8786.59, 0.0, 232227.6, 0.0, 0.0, 15, 8815.09, -9635.71, 0.0, 6755, 8786.59, 4, 0, 398.82, 0, 0, 274901.32, -1276, None, 26663.38, 31176.66, 99.48, 24336.99, 4.28, 305484.53, 0.0, 0.0, 1, 0.0, -44919.57, 3.23, 7136, 24336.99, 13, 0, 1466.25)]
Count:4735593


## Calcular clase ternaria

In [18]:
# Crear campos auxiliares
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe1 BOOLEAN;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe2 BOOLEAN;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS clase_ternaria STRING;
"""
do_sql(sql)




ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe1 BOOLEAN;

[]

ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe2 BOOLEAN;

[]

ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS clase_ternaria STRING;

[]


In [19]:
# existe1
sql="""
UPDATE competencia_02 c1
SET existe1 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 1)
);
"""
do_sql(sql)


UPDATE competencia_02 c1
SET existe1 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 1)
);

[(4735593,)]


In [20]:
# existe 2
sql="""
UPDATE competencia_02 c1
SET existe2 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 2)
);
"""
do_sql(sql)


UPDATE competencia_02 c1
SET existe2 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 2)
);

[(4735593,)]


In [21]:
# clase ternaria
sql="""
UPDATE competencia_02
SET clase_ternaria = CASE
    WHEN existe2 = TRUE THEN 'CONTINUA'
    WHEN existe1 = FALSE THEN 'BAJA+1'
    WHEN existe1 = TRUE AND existe2 = FALSE THEN 'BAJA+2'
END;
"""
do_sql(sql)


UPDATE competencia_02
SET clase_ternaria = CASE
    WHEN existe2 = TRUE THEN 'CONTINUA'
    WHEN existe1 = FALSE THEN 'BAJA+1'
    WHEN existe1 = TRUE AND existe2 = FALSE THEN 'BAJA+2'
END;

[(4735593,)]


In [22]:
# elimina auxiliares
sql="""
ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe1;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe2;
"""
do_sql(sql)


ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe1;

[]

ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe2;

[]


## Graba output

In [ ]:
df = conn.execute("SELECT * FROM competencia_02").fetchdf()



In [ ]:


# Grabar como CSV comprimido
output_file_path =os.path.join(dataset_path, output_file)
df.to_csv(output_file_path, index=False, compression='gzip')


In [27]:
# Cerrar la conexión cuando termines
conn.close()